In [9]:
import numpy as np
import pandas as pd
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')

In [76]:
df = pd.read_csv('data/Cleaned.csv')
cleaned = pd.read_csv('data/Cleaned.csv')

(0.0, 25.0]     1340
(25.0, 50.0]    1329
(50.0, inf]      309
nan               53
Name: AgeBin, dtype: int64

In [78]:
df = df[~df['Profession'].isin(['Others'])]
# df.Profession.value_counts()
countries = ['China', 'Mexico', 'Bangladesh', 'United States','Malaysia','Pakistan','Zambia','Japan']
# cleaned =df[df['Country'].isin(countries)]
cleaned = df
cleaned = cleaned[~((cleaned['Country'].isin(['Bangladesh','Pakistan'])) & (cleaned['Age'] > 50))]
cleaned = cleaned[~((cleaned['Country'].isin(['Mexico'])) & (cleaned['Education'] =='P'))]
bins = pd.IntervalIndex.from_tuples([(0,25),(25,50),(50,np.inf)])
cleaned['AgeBin'] = pd.cut(df.Age,bins)
cleaned.AgeBin = cleaned.AgeBin.astype(str)
cleaned.AgeBin.value_counts()
cleaned.shape

(2945, 84)

In [79]:
params = ['AgeBin','Education','Profession','Sex']

In [80]:
def func(df,params):
    freq = df.groupby(['Country',params[0]])[params[0]].agg(['count'])
    freq.columns = ['f']
    freq = freq.reset_index()
    freq = freq.pivot(index=params[0],columns="Country")
    for i in params[1:]:
        temp = df.groupby(['Country',i])[params[0]].agg(['count'])
        temp.columns = ['f']
        temp = temp.reset_index()
        temp = temp.pivot(index=i,columns="Country")
        freq = pd.concat([freq,temp])
        
    return freq
        

In [81]:
temp = func(cleaned,params).T
temp = temp.fillna(0).astype(int)

In [82]:
temp.to_csv('Country_Table.csv')

In [83]:
temp = pd.read_csv('Country_Table.csv')

In [85]:
temp[temp.Country.isin(countries)].to_csv('Freq_Table.csv',index=False)